In [1]:
import openai
import time
# openai.api_key = 's__k-73ZF1jNfuykC7q3L1wL4T3BlbkFJRRPFp17Qlg9Y9eaFJ32___r'


In [2]:
topic    = "acquisition of company named XYZ"

In [3]:
def parse_text(text):
    sub_topics = text.strip().split("\n")
    sub_topics_list = [sub_topic.strip().split(".", 1)[1].strip() for sub_topic in sub_topics if sub_topic]
    return sub_topics_list

def get_subtopics(topic):
    additional_infos_prompt = f"give me 5 sub-topics related to {topic} without explanations and be generic"
    print(additional_infos_prompt)
    res_additional_prompt = openai.Completion.create(
      engine="text-davinci-003",
      prompt=additional_infos_prompt,
      max_tokens=2000,
      temperature=0.7,
      stop=None,
    )
    additions = parse_text(res_additional_prompt['choices'][0]['text'])
    print(additions)
    return additions

In [4]:
additions = [ x for x in get_subtopics(topic) if x != '']

give me 5 sub-topics related to acquisition of company named XYZ without explanations and be generic
['Acquisition Financing', 'Merger & Acquisition Strategies', 'Legal Considerations', 'Integration of Businesses', 'Value Creation']


In [ ]:
#= "legal terms and random purchase price"
# all_res = []
# for addition in additions:
#     source   = "an internal corporate email"
#     N_per_request = 3
#     prompt =  f"""write {N_per_request} versions for a paragraph about {topic}. 
#               focus about {addition}. 
#               assume the content is taken from {source}
#               no paragraph numbering"""
#     print(prompt)
#     try:
#         res = openai.Completion.create(
#           engine="text-davinci-003",
#           prompt=prompt,
#           max_tokens=2000,
#           temperature=0.7,
#           stop=None,
#         )
#     except Exception as e:
#         print(e)
        
        
#     print(res['choices'][0]['text'])
#     all_res.append(res)
#     time.sleep(5)
    

In [5]:
additions

['Acquisition Financing',
 'Merger & Acquisition Strategies',
 'Legal Considerations',
 'Integration of Businesses',
 'Value Creation']

In [6]:
all_res = []
for addition in additions:
    source   = "an internal corporate email"
    N_per_request = 5
    prompt =  f"""write a short paragraph about the {addition} with regard to {topic}. 
              assume the content is taken from {source}"""
    print(prompt)
    try:
        res = openai.Completion.create(
          engine="text-davinci-003",
          prompt=prompt,
          max_tokens=2000,
          temperature=0.7,
          n=N_per_request,
          presence_penalty=0.2,
          stop=None,
        )
    except Exception as e:
        print(e)
        
        
    print([x['text'] for x in res['choices']])
    all_res.append([x['text'] for x in res['choices']])
    time.sleep(1)
    

write a short paragraph about the Acquisition Financing with regard to acquisition of company named XYZ. 
              assume the content is taken from an internal corporate email
['\n\nWe are pleased to announce that we have completed the Acquisition Financing for the acquisition of XYZ. This financing was secured through a combination of equity and debt investments, giving us the funds needed to complete the acquisition. We would like to thank all of our investors and partners who made this possible. We look forward to the new opportunities that this acquisition will bring.', '\n\nThe Acquisition Financing for the purchase of XYZ is now complete. We have successfully secured additional capital to fund the purchase of the company. Our team has worked diligently to ensure that this financing was completed in a timely and cost-effective manner. With the acquisition of XYZ, we are now in a better position to expand our operations and increase our market reach.', '\n\nAcquisition Financi

['\n\nWe are pleased to announce that XYZ has been successfully acquired by our company. This acquisition marks an important milestone in our business integration process, allowing us to expand our business operations in the XYZ market. XYZ will help our company strengthen our presence in the region and increase our market share. We look forward to leveraging the expertise of the XYZ team to create a more comprehensive and integrated business solution.', "\n\nThe integration of Businesses with regard to the acquisition of company XYZ is progressing well. Our teams have been working diligently to ensure a smooth transition for both companies, and we anticipate that XYZ's employees and customers will be pleased with the results. We are confident that this acquisition will provide us with a significant competitive advantage in the market and serve as a strong foundation for our continued success.", "\n\nIntegrating the newly acquired company, XYZ, into our business will be a critical part

In [7]:
import xlsxwriter
 
# Workbook() takes one, non-optional, argument
# which is the filename that we want to create.
workbook = xlsxwriter.Workbook('data.xlsx')
 
# The workbook object is then used to add new
# worksheet via the add_worksheet() method.
worksheet = workbook.add_worksheet()
 
# Use the worksheet object to write
row = 0
for addition_ind, addition_res in enumerate(all_res):
    for text in addition_res:
        worksheet.write(row,0, topic)
        worksheet.write(row,1, additions[addition_ind])
        worksheet.write(row,2, text)
        row+=1
 
# Finally, close the Excel file
# via the close() method.
workbook.close()